In [51]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import pandas as pd
from bs4 import BeautifulSoup

In [5]:
browser=webdriver.Chrome()

In [6]:
browser.get('https://www.tripadvisor.in/Hotels-g186338-London_England-Hotels.html')


In [7]:
accept = browser.find_element(By.ID, "onetrust-accept-btn-handler")
accept.click()

In [8]:
bsobj = BeautifulSoup(browser.page_source, 'lxml')

In [18]:
hotels = []
for name in bsobj.findAll('div',{'class':'listing_title'}):
  hotels.append(name.text.strip())
len(hotels)

33

In [55]:
ratings = []
for rating in bsobj.findAll('a',{'class':'ui_bubble_rating'}):
      if rating['alt'] == "5 of 5 bubbles":
            rating = 10
      elif rating['alt'] == "4.5 of 5 bubbles":
            rating = 9
      elif rating['alt'] == "4 of 5 bubbles":
            rating = 8
      elif rating['alt'] == "3.5 of 5 bubbles":
            rating = 7
      else:
            rating = 6
      ratings.append(rating)
len(ratings)

33

In [38]:
reviews = []
for review in bsobj.findAll('a',{'class':'review_count'}):
  reviews.append(review.text.replace('reviews','').strip())
len(reviews)

33

In [33]:
price = []
for p in bsobj.findAll('div',{'class':'price-wrap'}):
  price.append(p.text.replace('€','').strip()) 
price[:5]
len(price)

33

In [60]:
d1 = {'Hotel':hotels,'Ratings':ratings,'No_of_Reviews':reviews,'Price, €':price}
df = pd.DataFrame.from_dict(d1)

In [61]:
df

,Hotel,Ratings,No_of_Reviews,"Price, €"
0,Sponsored YOTEL London Shoreditch,9,684,112
1,1. The Montcalm Royal London House,10,"3,351",208
2,2. Park Grand London Hyde Park,8,"3,362",144
3,3. Park Grand Paddington Court,9,"3,206",185142
4,4. The Resident Covent Garden,10,"1,241",295
5,5. Travelodge London City hotel,9,"2,376",59
6,"Sponsored The Biltmore Mayfair, LXR Hotel...",8,173,930
7,6. Canopy by Hilton London City,10,420,195
8,7. Hard Rock Hotel London,9,"2,231",388
9,8. The Montcalm London Marble Arch,10,"3,810",477


In [15]:
df.to_csv("tripadvisor_london_list.csv")